In [43]:
import selenium
import json
import time
import re
import string
import requests
import bs4
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


In [44]:
#This is the English version of the booking domain
domain =  'https://www.booking.com/index.html?lang=en-us'


In [45]:
def prepare_driver(url):
    '''Returns a Firefox Webdriver.'''
    options = Options()
    #options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ss')))
    return driver


In [46]:
def fill_form(driver, search_argument):
    '''Finds all the input tags in form and makes a POST requests.'''
    search_field = driver.find_element_by_id('ss')
    search_field.send_keys(search_argument)
    # We look for the search button and click it
    driver.find_element_by_class_name('sb-searchbox__button').click()
    wait = WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sr-hotel__title')))

In [47]:
def scrape_results(driver, n_results):
    '''Returns the data from n_results amount of results.'''

    accommodations_urls = list()
    accommodations_data = list()

    for accomodation_title in driver.find_elements_by_class_name('sr-hotel__title'):
        accommodations_urls.append(accomodation_title.find_element_by_class_name('hotel_name_link').get_attribute('href'))

    for url in range(0, n_results):
        if url == n_results:
            break
        url_data = scrape_accommodation_data(driver, accommodations_urls[url])
        accommodations_data.append(url_data)

    return accommodations_data

In [48]:
def scrape_accommodation_data(driver, accommodation_url):
    '''Visits an accommodation page and extracts the data.'''

    if driver == None:
        driver = prepare_driver(accommodation_url)

    driver.get(accommodation_url)
    time.sleep(5)

    accommodation_fields = dict()

    # Get the Hotel name to print
    hotel_name = driver.find_element_by_id('hp_hotel_name').text
    print(hotel_name + "\n=====================================\n")

    #Show the reviews of the hotel
    driver.find_element_by_id('show_reviews_tab').click()
    time.sleep(5)

    # find the number of pages with the reviews results
    rev_pages = int (driver.find_element_by_xpath("/html/body/div[5]/div/div[3]/div[2]/div[1]/div[11]/div/div[1]/div[2]/div[2]/div[3]/div[3]/div[4]/div/div[1]/div/div[2]/div/div[7]/a/span[1]").text)
    print ("The total number of pages with reviews is: " + str(rev_pages) + "\n\n")

    #First Page of reviews
    print("FIRST PAGE-------------------------------------\n")
    reviews = driver.find_elements_by_class_name('c-review__body') #this is the first page
    ii=0
    for rev in reviews:
        print (str(ii+1) + ". " + rev.text)
        ii = ii+1

    # Second page of reviews
    for ind in range (2,rev_pages+1):
        rev_link = driver.find_element_by_class_name('pagenext').get_attribute('href')
        driver.get(rev_link)
        time.sleep(5)
        print("PAGE " + str(ind) + " -------------------------------------\n")
        reviews = driver.find_elements_by_class_name('c-review__body')
        ii=0
        for rev in reviews:
            print (str(ii+1) + ". " + rev.text)
            ii=ii+1



    '''
    # Get the accommodation name
    accommodation_fields['name'] = driver.find_element_by_id('hp_hotel_name').text.strip('Hotel')

    # Get the accommodation score
    accommodation_fields['score'] = driver.find_element_by_class_name('bui-review-score--end').find_element_by_class_name('bui-review-score__badge').text

    # Get the accommodation location
    accommodation_fields['location'] = driver.find_element_by_id('showMap2').find_element_by_class_name('hp_address_subtitle').text

    # Get the most popular facilities
    accommodation_fields['popular_facilities'] = list()
    facilities = driver.find_element_by_class_name('hp_desc_important_facilities')

    for facility in facilities.find_elements_by_class_name('important_facility'):
        accommodation_fields['popular_facilities'].append(facility.text)
    '''
    return accommodation_fields



In [49]:
if __name__ == '__main__':

    try:
        driver = prepare_driver(domain)
        fill_form(driver, 'Delphi')
        accommodations_data = scrape_results(driver, 1)
        accommodations_data = json.dumps(accommodations_data, indent=4)
        with open('booking_data.json', 'w') as f:
            f.write(accommodations_data)
    finally:
        driver.quit()


Hotel Fedriades Delphi Hotel

The total number of pages with reviews is: 80


FIRST PAGE-------------------------------------

1. Everything was beautiful, 4th floor room had an amazing view. Staff friendly and everything clean. Included breakfast was delicious.
2. No complaints
3. the great location close to souvenirs shops and restaurants, beautiful view on the valley, friendly and efficient staff , breakfast according wishes ( choice from a menu on arrival), all the hygiene rules strictly respected, the interconnected double rooms offer, nice breakfast room and lobby
4. This is a lovely, clean property which is easy walking distance to the Delphi historic site. The rooms are clean and comfortable. There are also lots of different restaurants in easy walking distance. We found it easy to park our car out front.
5. Nothing.
6. the view is very nice and the staff very friendly. It's located near the archeological site. 
They handled the actual situation professional.
7. Fantastic views

PAGE 5 -------------------------------------

1. Really friendly staff, clean room, comfy bed and a fantastic view from the balcony.
2. Second time we stay at Fedriades, and we were rewarded with the same great service and hospitality as last time. The room was just right, the bed comfy and the aircon effective, although a bit noisy. The room, bathroom and hotel are very clean, and the staff very helpful. There is a small car park, be sure to reserve a space if you need one as it is hard to find one otherwise in the town should you be there during the peak season. The hotel also offer a 10% discount with a local taverna, this is highly recommended as there is a very good selection of local foods. The hotel's location means you can walk to the Delphi archaeological site without having to drive, as well as to any other part of the town, which is very convenient.
3. The view was great and the room really comfortable.
4. It would have been great to check in immediately upon arrival but I d

PAGE 10 -------------------------------------

1. The location is great for visiting the archeological site and museum. Free parking across the street. Nice view of the Gulf of Corinth from the room. The lobby has a spacious but cosy seating area, great for lounging around in. The receptionist Anna was so warm, friendly and helpful - she is a great asset to the hotel. She’s the best!
2. Shower within a bathtub didn’t have a door or curtain, which made the floor very wet after each use. No hot cooked food for the breakfast buffet.
3. We loved the view, the very comfortable bed and also the staff was very friendly and pleasant.
4. Location Great Breakfast
5. Perfect View, quiet room. The Breakfast was really good too.
6. Surprisingly... our room is on the 4th floor with special elevator entrance... wow.. didn't realize it when booking... room is specious for 3 of us and the view from the room is magnificent... If you 1st timer and probably this is 1st and last time visiting Delphi.. book

PAGE 14 -------------------------------------

1. Location
2. Construction work during the day Poor breakfast Photos are heavily enhanced and do not show real condition of hotel
3. Rented 2 rooms, we had the junior suite which was the only suite in the hotel. It was nice, enjoyed the extra bit of space. Nice bathroom and linens.
4. The management was rude, had to park the car 3 times in order to get the correct spot. The second room was small and smelled like cigarettes our guests were uncomfortable but they made due for one night. The breakfast was pretty bad lots of processed meats, hard boiled eggs, warm milk & juices, defrosted Vienna sausages and wrapped breading were the highlight:/ They have been bugging us about a electronic key they are claiming we did not return; all electronic keys were returned in the morning to the morning cook/host. No ice. Management had the personality of a wet towel.
5. Location
6. Disappointing room (pictures looked better ) disappointing panoramic vi

PAGE 18 -------------------------------------

1. Η θέα από το δωμάτιο ήταν μαγική!Είναι στην καρδιά των Δελφών! Πολύ πρόθυμο το προσωπικό να δώσει την καλύτερη καθοδήγηση σε ότι χρειαστήκαμε!Το πρωινό εξαιρετικό! Σίγουρα θα το επισκεφτούμε ξανά!
2. 
3. Персоналът е любезен, чисто е, гледката е красива
4. 
5. Икономичната стая е много малка, тясно е
6. 
7. Αψογο προσωπικο Πλουσιο πρωινο Υπεροχη θεα από το δωματιο Ανετη μπανιερα με υδρομασαζ
8. 
9. Μειναμε στη σουιτα, ο θορυβος απο το ασανσερ ηταν λιγο ενοχλητικος.
10. 
11. Πολύ ωραία τοποθεσία με απεριόριστη θέα στη θάλασσα. Το δωμάτιο ήταν μοντέρνο και πρακτικό, ενώ διέθετε όλες τις απαραίτητες παροχές. Υπήρχε και βραστήρας, στον οποίο μπορούσες να ετοιμάσεις τσάι και καφέ. Το κρεβάτι ήταν πολύ άνετο. Το πρωινό ήταν πλούσιο, γευστικότατο (η ομελέτα καταπληκτική) και ικανοποιούσε όλα τα γούστα.
12. 
13. Η τηλεόραση ήταν πολύ μικρή και τοποθετημένη ψηλά στον τοίχο δίπλα στη γωνία. Μικρής σημασίας παρατήρηση, καθώς η θέα από το δωμάτιο σ

...

32. 
33. Al ser un pueblo pequeñito, está cerca de todo. Tiene muchas tiendas cercas ideales para comprar regalos porque el precio es muy inferior al que encuentras en Atenas o las islas. El hotel es cómodo y el personal agradable. Hay buenos bares cercanos también. La habitación cómoda, aunque el baño un poco antiguo. Buenas vistas.
34. 
35. Nada especialmente reseñable
36. 
PAGE 42 -------------------------------------

1. Petit déjeuner exceptionnel. Le lit très confortable. Ascenseur. La climatisation peu bruyante. Personnel sympathique.
2. 
3. Soyez à l'heure pour le petit déjeuner (7h-10h). A 9h55, il commence déjà à ranger malgré 3 tables en train de déjeuner.
4. 
5. abbiamo prenotato la suite , e la mia famiglia è rimasta contentissima per, L'ASCENSORE CHE ARRIVAVA DIRETTAMENTE IN SUITE, PER LA VASCA IDRO, PER IL CONFORT DEI LETTI, E L'OTTIMA VISTA . TUTTO ECCELLENTE.
6. 
7. Ottima posizione con camere vista mare e vallata.
8. 
9. Da migliorare un po’ il bagno e i serramenti.
1

38. 
39. Das Personal an der Rezeption könnte höflicher sein. Die Ausstellung ist eine Sammlung verschiedenster alter Möbel. Es gibt keine internationalen Nachrichten Sender im Fernseher lediglich regionale sender.
40. 
PAGE 47 -------------------------------------

1. 호텔 위치는 델피 초입에 있어서 아폴로 성역까지는 10분 정도 걸어야 합니다. 델피 타운 자체가 끝까지 걸어서 채 5분이 걸리지 않을 만큼 작아서 불편하지는 않았어요. 커넥팅 룸을 신청했는데 욕실이 2개여서 편리하게 사용했어요.
2. 
3. 주차 가능이었는데 그냥 길 가 아무데나 차를 세우는 거였어요. 호텔 직원에게 문의했는데 알아서 차를 세워라 식이어서 매우 당황했습니다. 조식은 먹을 것이 너무 없어서 굳이 신청하지 않아도 될 것 같아요.
4. 
5. Emplacement
6. 
7. Très bruyant
8. 
9. La foto della camera quadrupla n 101 presenta le tende chiuse, per cui non vi attende il balconcino con sedie e vista panoramica, ma solo la finestra del primo piano con affaccio su strada e vista parziale per i tetti . Inoltre, la camera è l' unica che presenta l'uscita nella sala colazione con tavoli e sedie .
10. 
11. Lo mejor sin duda el desayuno. Las habitaciones normalitas. Cogimos dos y en una de ellas el water estaba metido

PAGE 51 -------------------------------------

1. Так называемый "панорамный вид" больше похож на боковой
2. 
3. Lo tranquilo de sus alrededores
4. 
5. Quizás la almohada un poco blanda y el desayuno caro (8 euros)
6. 
7. Posizione
8. 
9. La vista dava su un muro e la colazione non era buona. Il succo di frutta all’arancia era frizzante...
10. 
11. Posizione, pulizia, gentilezza e disponibilità dello staff, panorama.
12. 
13. Nulla
14. 
15. Bonne situation pour aller aux sites archéologiques. Chambre d un très bon rapport qualité prix Petit déjeuner appétissant et varié.
16. 
17. Locatie! Loopt zo naar Delphi en de restaurants.
18. 
19. Geen buitenruimte om te relaxen.
20. 
21. Todo muy bien
22. 
23. La localisation et le fait d’avoir été upgradé à notre arrivée.
24. 
25. L’isolation quasi inexistante avec le bruit dans les couloirs
26. 
27. L’emplacement
28. 
29. WIFI très mauvais signal dans les chambres Petit déjeuner cher et pas bon propreté limite. Cheveux trouvés dans la salle de

PAGE 65 -------------------------------------

1. There are no comments available for this review
2. There are no comments available for this review
3. There are no comments available for this review
4. There are no comments available for this review
5. There are no comments available for this review
6. There are no comments available for this review
7. There are no comments available for this review
8. There are no comments available for this review
9. There are no comments available for this review
10. There are no comments available for this review
PAGE 66 -------------------------------------

1. There are no comments available for this review
2. There are no comments available for this review
3. There are no comments available for this review
4. There are no comments available for this review
5. There are no comments available for this review
6. There are no comments available for this review
7. There are no comments available for this review
8. There are no comments available for

PAGE 80 -------------------------------------

1. There are no comments available for this review
2. There are no comments available for this review
3. There are no comments available for this review
4. There are no comments available for this review
5. There are no comments available for this review
6. There are no comments available for this review
